Tests

In [ ]:
%pip install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Randomly generate grayscale image data (28x28 like MNIST)
sample_images = np.random.rand(5, 28, 28)  # 5 images of 28x28 pixels

fig, axes = plt.subplots(1, 5, figsize=(10, 3))
for i in range(5):
    axes[i].imshow(sample_images[i], cmap="gray")
    axes[i].axis("off")
plt.show()


In [ ]:
%pip install tensorflow
%pip install tensorflow-metal
%pip install keras_cv matplotlib diffusers transformers torch scipy


In [ ]:
import tensorflow as tf

# Check if TensorFlow detects the GPU
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Devices:", tf.config.list_physical_devices('GPU'))

# Check if TensorFlow is running on Metal
tf.debugging.set_log_device_placement(True)

# Force TensorFlow to run on GPU if available
device = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"
print("Using device:", device)

# Perform a simple matrix multiplication to see if it runs on GPU
import time
with tf.device(device):
    a = tf.random.normal([1000, 1000])
    b = tf.random.normal([1000, 1000])
    
    start = time.time()
    c = tf.matmul(a, b)
    print("Computation time To do a 1000x1000 matrix multiplication:", time.time() - start)


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt

# Initialize Stable Diffusion model
model = keras_cv.models.StableDiffusion(img_width=512, img_height=512)

# Generate images from text prompt
images = model.text_to_image("A futuristic city at sunset", batch_size=1)

# Display images
def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")
    plt.show()

plot_images(images)


In [ ]:
!pip3 install torch torchvision torchaudio -f https://download.pytorch.org/whl/metal.html # latest stable

In [ ]:
import torch

print(torch.backends.mps.is_available())  # Should return True


In [ ]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

# Check if MPS (Metal Performance Shaders) is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Define a simple transformer-based language model
class SimpleTransformerLM(nn.Module):
    def __init__(self, vocab_size=10000, embedding_dim=512, nhead=8, 
                 num_layers=6, dim_feedforward=2048):
        super(SimpleTransformerLM, self).__init__()
        
        # Word embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Positional encoding (simplified for this example)
        self.pos_encoder = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.GELU()
        )
        
        # Transformer encoder layers
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=embedding_dim, 
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        # Output layer
        self.output = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, x):
        # x shape: [batch_size, seq_len]
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.output(x)  # [batch_size, seq_len, vocab_size]
        return x

# Create model and move to MPS device
vocab_size = 10000  # Vocabulary size
seq_length = 128    # Sequence length
batch_size = 16     # Batch size

model = SimpleTransformerLM(vocab_size=vocab_size)
model.to(device)  # Move model to GPU
print(f"Model moved to {device}")

# Generate some dummy data
input_data = torch.randint(0, vocab_size, (batch_size, seq_length)).to(device)
target_data = torch.randint(0, vocab_size, (batch_size, seq_length)).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model for a few steps to demonstrate GPU usage
def train_step(model, inputs, targets):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(inputs)
    # Reshape for cross entropy
    outputs = outputs.view(-1, vocab_size)
    targets = targets.view(-1)
    
    # Calculate loss
    loss = criterion(outputs, targets)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    return loss.item()

# Test training performance
print("\nTraining for 20 iterations to test performance:")
start_time = time.time()
for i in range(20):
    loss = train_step(model, input_data, target_data)
    print(f"Iteration {i+1}, Loss: {loss:.4f}")
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds") # 2.80 sec on GPU, 13.35 sec on CPU (MacBook Pro M2 16GB)

# Generate predictions with the model
def generate_text(model, start_tokens, max_length=20):
    model.eval()
    current_tokens = start_tokens.clone()
    
    for _ in range(max_length):
        with torch.no_grad():
            # Get model predictions for the next token
            logits = model(current_tokens)
            next_token_logits = logits[:, -1, :]
            
            # Sample from the distribution
            probs = torch.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, 1)
            
            # Append new token to sequence
            current_tokens = torch.cat([current_tokens, next_token], dim=1)
    
    return current_tokens

# Try generating some "text" (just token IDs in this example)
start_seq = torch.randint(0, vocab_size, (1, 5)).to(device)
generated = generate_text(model, start_seq)
print("\nGenerated token sequence:")
print(generated.cpu().numpy())